In [1]:
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import csv

In [2]:
class MyDigitDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        image_paths = []
        for ext in ('*.jpg', '*.jpeg', '*.png'):
            image_paths.extend(glob.glob(os.path.join(root_dir, '*', 'hand_written_digit', '*', ext)))
        self.image_paths = image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = int(os.path.basename(img_path).split('_')[0])
        if self.transform:
            image = self.transform(image)
        return image, label

class MyTestDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.transform = transform
        self.image_paths = []
        for ext in ('*.jpg', '*.jpeg', '*.png'):
            self.image_paths.extend(glob.glob(os.path.join(folder, ext)))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        while idx < len(self.image_paths):
            img_path = self.image_paths[idx]
            try:
                image = Image.open(img_path).convert("RGB")
                if self.transform:
                    image = self.transform(image)
                return image, img_path
            except Exception:
                idx += 1  # Bỏ qua ảnh lỗi, thử ảnh tiếp theo
        raise IndexError("No valid image found at or after index {}".format(idx))

In [3]:
transformtrain = transforms.Compose([
    transforms.Resize((32, 32)),
     
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
transformtest =transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [4]:
root_path = r'C:\Users\Admin\Desktop\python\doan'
traindata = MyDigitDataset(root_path, transform=transformtrain)
dataloader = DataLoader(traindata, batch_size=64, shuffle=True, num_workers=0)

test_folder = r'C:\Users\Admin\Desktop\python\MUOINGAN'
testdata = MyTestDataset(test_folder, transform=transformtest)
test_loader = DataLoader(testdata, batch_size=64,num_workers=0, shuffle=False)

In [5]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}")

Epoch 1, Loss: 2.260616837715616
Epoch 2, Loss: 1.9641858059532789
Epoch 3, Loss: 1.5521157201455564
Epoch 4, Loss: 1.2415808244627349
Epoch 5, Loss: 1.0456505284017446
Epoch 6, Loss: 0.8932091903929807
Epoch 7, Loss: 0.7895599962497244
Epoch 8, Loss: 0.6794294836569805
Epoch 9, Loss: 0.5888341020564644
Epoch 10, Loss: 0.5045538885252816
Epoch 11, Loss: 0.4489511950891845
Epoch 12, Loss: 0.3977263816157166
Epoch 13, Loss: 0.33197792695493117
Epoch 14, Loss: 0.29135203863285025
Epoch 15, Loss: 0.26279110970849895
Epoch 16, Loss: 0.22359778101042826
Epoch 17, Loss: 0.19507119131787698
Epoch 18, Loss: 0.17654724076998476
Epoch 19, Loss: 0.16537073866597243
Epoch 20, Loss: 0.1616534200706044


In [8]:
model.eval()
results = []
with torch.no_grad():
    for images, img_paths in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        preds = preds.cpu().numpy()
        for path, pred in zip(img_paths, preds):
            filename = os.path.basename(path)
            results.append([filename, int(pred)])

with open('WECODE_CNN_10k.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['filename', 'prediction'])
    writer.writerows(results)